In [47]:
import yaml
import numpy as np
import pandas as pd

from tabulate import tabulate
from IPython.display import display, clear_output

import matplotlib.pyplot as plt
from datetime import datetime

In [7]:
processed_unlabeled_transactions = pd.read_csv("processed_unlabeled_transactions.csv")
processed_accounts = pd.read_csv("processed_accounts.csv")
processed_users = pd.read_csv("processed_users.csv")
processed_users_metadata = pd.read_csv("processed_users_metadata.csv")
processed_organizations = pd.read_csv("processed_organizations.csv")          
processed_users_organizations = pd.read_csv("processed_users_organizations.csv")
processed_organizations_metadata = pd.read_csv("processed_organizations_metadata.csv")
processed_labeled = pd.read_csv("processed_labeled_transactions.csv")

/Users/ymquint/Documents/Economapas/economapas/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [17]:
def account_trail(account_id, age_selfie_date={}, recursion = False):
    """
    Args:
    
    account_id: string
    
    Return:
    age_selfie_date: dict, keys: age range, number of selfie, and first inseret at date
    
    
    In this function we consider to sum in each timestamp 3 hours (10800 seconds), this represents brazilian time 
    
    """
    if not recursion:
        owner_id = processed_accounts[processed_accounts.id == account_id].owner_id.values[0]
        owner_type = processed_accounts[processed_accounts.id == account_id].owner_type.values[0]
        age_selfie_date = {"account_id":account_id}
    else:
        
        owner_id = account_id
        owner_type = "user"
    
    if owner_type == "user":
        type_dada = processed_users_metadata[processed_users_metadata.user_id == owner_id].type
        value = processed_users_metadata[processed_users_metadata.user_id == owner_id].value
        inserted_at_user_metadata = [date.timestamp() + 10800 for date in 
                                     pd.to_datetime(processed_users_metadata[processed_users_metadata
                                                                             .user_id == owner_id].inserted_at)]
        for t_data, value in zip(type_dada, value):
            
            if t_data == "number_of_selfies_sent":
                age_selfie_date[t_data] = int(value)
            else:
                age_selfie_date[t_data] = value
        
    
        inserted_at_user = [date.timestamp() + 10800 for date in pd.to_datetime(processed_users[processed_users.id 
                                                          == owner_id].inserted_at)]
        
        oldest_date = datetime.now().timestamp() + 10800
        if len(inserted_at_user_metadata) > 1:
            for date in inserted_at_user_metadata:
                if oldest_date > date:
                    oldest_date = date
        if oldest_date > inserted_at_user[0]:
            oldest_date = inserted_at_user[0]
            
        age_selfie_date["oldest_inserted_at"] = oldest_date
        
        if len(age_selfie_date.keys()) != 4:
            for key in ["account_id", "age_range", "number_of_selfies_sent", "oldest_inserted_at"]:
                if key not in age_selfie_date.keys():
                    age_selfie_date[key] = np.nan
        
        return age_selfie_date
    
    else:
        inserted_at_organization_metadata = [date.timestamp() + 108100 for date in 
                                             pd.to_datetime(processed_organizations_metadata[processed_organizations_metadata.
                                                                              organization_id == owner_id].inserted_at)]
        if len(inserted_at_organization_metadata) == 0:
            inserted_at_organization_metadata = [datetime.now().timestamp()]
        
        inserted_at_organization = [date.timestamp() + 108100 for date in 
                                    pd.to_datetime(processed_organizations[processed_organizations.id ==
                                                                           owner_id].inserted_at)]
        
        if len(inserted_at_organization) == 0:
            inserted_at_organization = [datetime.now().timestamp()]
        
        
        user_id = processed_users_organizations[processed_users_organizations.organization_id ==
                                                owner_id].user_id.values[0]
        age_selfie_date = account_trail(user_id, age_selfie_date, recursion=True)

        if len(inserted_at_organization_metadata) > 2:
            for date in inserted_at_organization_metadata:
                if age_selfie_date["oldest_inserted_at"] > date:
                    age_selfie_date["oldest_inserted_at"] = date
        else:
            if age_selfie_date["oldest_inserted_at"] > inserted_at_organization_metadata[0]:
                age_selfie_date["oldest_inserted_at"] = inserted_at_organization_metadata[0]
                
        if len(inserted_at_organization) > 2:
            for date in inserted_at_organization:
                if age_selfie_date["oldest_inserted_at"] > date:
                    age_selfie_date["oldest_inserted_at"] = date
        else: 
            if age_selfie_date["oldest_inserted_at"] > inserted_at_organization[0]:
                age_selfie_date["oldest_inserted_at"] = inserted_at_organization[0]
        
        return age_selfie_date
        
    #depois tem que verificar se todos tem os dois age and selfie
            

In [18]:
def dict_trail_to_pandas(accounts_id):
    """
    Args:
    
    accounts_id, pd.Series, a series of accounts id
    
    Return:
    
    df, pd.DataFrame, return a df with the columns = [account_id, age_range, n_selfies, oldest_inserted_at]
    
    """
    len_account = len(accounts_id)
    len_account_unique = len(accounts_id.unique())
    
    print(f"Found {len_account} accounts, from this {len_account_unique} accounts are unique")
    
    accounts_age_selfie_date = []
    
    i = 0
    for account_id in accounts_id.unique():
        print(f"{i}/{len_account_unique} processed", end="\r")
        age_selfie_date = account_trail(account_id)
        accounts_age_selfie_date.append(age_selfie_date)
        i += 1
        
    print("\n----End----")
            
    return pd.DataFrame(accounts_age_selfie_date)

In [44]:
def test_merge(number_of_sample: int):
    """
    The ideia here is to search some random index and display all search in tables
    
    Args:
    
    number_of_sample
    
    Return:
    
    none
    """
    
    random_index = np.random.randint(0, len(new_processed_labeled), number_of_sample)
    i = 0
    
    for n in random_index:
        
        if i == 0:
            entry = processed_labeled.loc[random_index]
            display(entry)
            account_id = entry.account_id.values[i]
            find_account = input(f"Find account {account_id}? ")
        else:
            display(entry)
            
        if find_account.lower() == "y":
            account_entry = processed_accounts[processed_accounts.id == account_id]
            display(account_entry)
            owner_id = account_entry.owner_id.values[0]
            owner_type = account_entry.owner_type.values[0]
            find_owner = input(f"Find {owner_type} {owner_id}? ")
            if find_owner.lower() == "y":
                if owner_type == "user":
                    display(processed_users_metadata[processed_users_metadata.user_id == owner_id])
                    display(processed_users[processed_users.id == owner_id])
                else:
                    display(processed_users_organizations[processed_users_organizations.organization_id == 
                                                                 owner_id])
                    user_id_find = processed_users_organizations[processed_users_organizations.organization_id == 
                                                                 owner_id].user_id.values[0]
                    
                    display(processed_organizations_metadata[processed_organizations_metadata.organization_id == owner_id])
                    display(processed_organizations[processed_organizations.id == owner_id])
                    find_user = input(f"Find user {user_id_find} from organization {owner_id}? ")
                    if find_user.lower() == "y":
                        display(processed_users_metadata[processed_users_metadata.user_id == user_id_find])
                        display(processed_users[processed_users.id == user_id_find])
                                
                show_account_trail = input(f"Show account trail for account id {account_id} and merge dataframe? ")
                if show_account_trail == "y":
                    account_t = account_trail(account_id)
                    account_t["oldest_inserted_at"] = datetime.fromtimestamp(account_t["oldest_inserted_at"])
                    display(account_t)
                    display(new_processed_labeled[new_processed_labeled.account_id == account_id].head(2))
                    
        i += 1
        
        if i < len(entry.account_id.values):
            account_id = entry.account_id.values[i]
            find_account = input(f"Find next account? Account id {account_id}? This erase previous output.")
            clear_output(wait=True)
        else:
            print("--------------------------------END--------------------------------")
    

                    
            
        
        

In [46]:
display(processed_labeled.head(1))
display(processed_accounts.head(1))
display(processed_users_metadata.head(1))
display(processed_organizations_metadata.head(1))
display(processed_users_organizations.head(1))
display(processed_users.head(1))
display(processed_organizations.head(1))

,operation_id,request_id,account_id,device_id,counterparty_document,operation_type,amount,balance,requested_at,processed_at,is_fraud
0,37e105f1-003f-465b-8179-e7705b12a24d,c1820b88-ac54-444f-b9a0-70de30124d95,a7b4f041-ef6a-4e17-baf9-cd471ef0f484,ef1756b6-354c-4ea4-b175-4eba835b60a5,38e6f7c9-e935-4c7c-8915-ef923a5ca914,cash_out_type_1,0.000099,0.000167,2018-11-21 15:41:23,2018-11-21 15:41:23,NaN


,id,owner_id,owner_document,owner_type,inserted_at
0,ac8a1faa-3748-4ef7-b140-8fe818cdc974,d2e3e3b7-e98d-4ef3-8a51-83db807b9a94,5e998c51-53cc-48f0-aef4-45ec12a466be,user,2017-03-16 16:33:43.603803


,id,owner_id,owner_document,owner_type,inserted_at
0,ac8a1faa-3748-4ef7-b140-8fe818cdc974,d2e3e3b7-e98d-4ef3-8a51-83db807b9a94,5e998c51-53cc-48f0-aef4-45ec12a466be,user,2017-03-16 16:33:43.603803


,id,user_id,type,value,inserted_at,updated_at
0,5e820e59-2793-4aa8-be6e-95d547872de7,7ac5f4a4-e46f-4d1a-b530-50d1930d9b95,age_range,40-59,2018-05-24 12:32:21.406260,2018-05-24 12:32:21.406260


,id,organization_id,type,value,inserted_at,updated_at
0,8a442911-65db-42a3-8063-2e8e2ddab984,417b2d7c-8a26-483b-920a-cc283095d044,user_is_partner,"{""mei"": false, ""partner"": true, ""user_id"": ""f3...",2018-11-25 02:31:05.472505,2018-11-25 02:31:05.472505


,user_id,organization_id
0,475c1048-860c-49e7-b94a-eb478eb78cf6,e4ef0830-64e3-4cae-bc14-a718849b60b7


,id,document,inserted_at
0,659945ba-eb5b-4112-b338-8284aec5d99b,96e7c1d9-8482-4bd6-abe7-90ad0b9854eb,2018-04-24 19:40:00.914691


,id,document,inserted_at
0,4f680da2-7b7c-4d48-9151-6f49437b4c27,9487f858-03b5-4eb4-afda-3e472643ef32,2018-11-01 17:40:15.194651


In [52]:
unique_value_org = processed_organizations_metadata.value.unique()

In [62]:
unique_values = {}
list_value = []
for value in unique_value_org: 
    try:
        value_load = yaml.safe_load(value)
        for key in value_load.keys():
            if key not in unique_values.keys() and key != "user_id":
                unique_values[key] = [value_load[key]]
            elif key != "user_id":
                if value_load[key] not in unique_values[key]:
                    unique_values[key].append(value_load[key])
    except:
        if value not in list_value:
            list_value.append(value)
print(unique_values)
print(list_value)

{'mei': [False, True], 'partner': [True, False], 'eireli': [False, True]}
['ME', nan, 'EPP', 'Other', 'Not informed']


In [3]:
data_set_len = len(processed_accounts)
id_len = len(processed_accounts.id.unique())
owner_id_len = len(processed_accounts.owner_id.unique())
owner_documentt_len = len(processed_accounts.owner_document.unique())

print("Dataset Len", data_set_len)
print("id_len_unique", id_len)
print("owner_id_len_unique", owner_id_len)
print("owner_documentt_len_unique", owner_documentt_len)



Dataset Len 293002
id_len_unique 293002
owner_id_len_unique 292889
owner_documentt_len_unique 291710


In [4]:
processed_users_metadata[processed_users_metadata.type == 'number_of_selfies_sent'].value.unique()

array(['5', '6', '11', '0', '4', '3', '12', '10', '15', '7', '1', '2',
       '8', '17', '13', '20', '21', '14', '22', '18', '16', '48', '25',
       '23', '9', '39', '35', '44', '26', '27', '32', '28', '30', '42',
       '29', '31', '40', '57', '24', '55', '50', '19', '41', '38', '34',
       '43', '36', '33', '61', '45', '56'], dtype=object)

In [58]:
processed_users_metadata[processed_users_metadata.value == "61"]

,id,user_id,type,value,inserted_at,updated_at
92317,47f206ed-65ce-45e4-b72f-ce86c40b6f1f,c015ec1e-fe3a-4940-927c-4988c6284375,number_of_selfies_sent,61,2018-03-13 02:55:50.065873,2018-03-13 02:55:50.065873


In [59]:
processed_accounts[processed_accounts.owner_id == "c015ec1e-fe3a-4940-927c-4988c6284375"]

,id,owner_id,owner_document,owner_type,inserted_at


In [16]:
processed_users_metadata.type.unique()

array(['age_range', 'number_of_selfies_sent'], dtype=object)

In [27]:
len(pd.to_datetime(processed_users_metadata[processed_users_metadata.user_id == "7ac5f4a4-e46f-4d1a-b530-50d1930d9b95"].inserted_at.values))

2

In [11]:
#grab some coffee

df = dict_trail_to_pandas(processed_labeled.account_id)

Found 940935 accounts, from this 63723 accounts are unique
63722/63723 processed
----End----


In [12]:
df.dtypes

account_id                 object
age_range                  object
number_of_selfies_sent    float64
oldest_inserted_at        float64
dtype: object

In [13]:
df.head()

,account_id,age_range,number_of_selfies_sent,oldest_inserted_at
0,a7b4f041-ef6a-4e17-baf9-cd471ef0f484,40-59,5.0,1.541947e+09
1,ed310df0-6834-4899-811e-5faf13b2a99c,>= 60,5.0,1.530573e+09
2,9b953a73-244b-45d9-a645-c86c7d455340,25-39,6.0,1.527039e+09
3,1db936d9-608e-4ea0-a749-2b2482f5caf2,25-39,5.0,1.538587e+09
4,7a4bc7e0-9cd1-41ab-a5c8-791993a81a6c,25-39,5.0,1.528404e+09


In [15]:
len(df)

63723

In [16]:
processed_labeled.dtypes

operation_id              object
request_id                object
account_id                object
device_id                 object
counterparty_document     object
operation_type            object
amount                   float64
balance                  float64
requested_at              object
processed_at              object
is_fraud                  object
dtype: object

In [17]:
len(processed_labeled)

940935

In [18]:
new_processed_labeled = processed_labeled.join(df.set_index("account_id"), on="account_id")


In [19]:
len(new_processed_labeled)

940935

In [20]:
new_processed_labeled.head()

,operation_id,request_id,account_id,device_id,counterparty_document,operation_type,amount,balance,requested_at,processed_at,is_fraud,age_range,number_of_selfies_sent,oldest_inserted_at
0,37e105f1-003f-465b-8179-e7705b12a24d,c1820b88-ac54-444f-b9a0-70de30124d95,a7b4f041-ef6a-4e17-baf9-cd471ef0f484,ef1756b6-354c-4ea4-b175-4eba835b60a5,38e6f7c9-e935-4c7c-8915-ef923a5ca914,cash_out_type_1,9.932046e-05,0.000167,2018-11-21 15:41:23,2018-11-21 15:41:23,NaN,40-59,5.0,1.541947e+09
1,72d66c19-130b-4d51-8ed7-e3b6c0b65034,9400a9f4-6ab8-40cd-b806-1569b693a88e,ed310df0-6834-4899-811e-5faf13b2a99c,ef1756b6-354c-4ea4-b175-4eba835b60a5,9bd89feb-e952-47c1-bbf5-d4c17b36d852,cash_out_type_3,1.285222e-05,0.000533,NaN,2018-10-22 23:43:38,NaN,>= 60,5.0,1.530573e+09
2,31abd386-09f7-4f18-9b01-d2682e3c2c65,80adf85b-56cc-4730-903d-e63c75ae839a,ed310df0-6834-4899-811e-5faf13b2a99c,ef1756b6-354c-4ea4-b175-4eba835b60a5,9bd89feb-e952-47c1-bbf5-d4c17b36d852,cash_out_type_3,1.582566e-05,0.000347,2018-11-19 16:11:57,2018-11-19 16:12:00,NaN,>= 60,5.0,1.530573e+09
3,27543733-e8a8-4593-88fb-921e205a6e0c,6edfea5c-4125-49fa-bbe7-b39433f6e49a,ed310df0-6834-4899-811e-5faf13b2a99c,ef1756b6-354c-4ea4-b175-4eba835b60a5,e573e2d3-39af-4f71-97df-bf400e854a8a,cash_out_type_3,5.672962e-07,0.000572,NaN,2018-10-29 15:20:49,NaN,>= 60,5.0,1.530573e+09
4,6aff8f2d-0730-4f94-846b-14c059550ab2,88ddff46-6714-4a47-9407-a7d54996dde1,ed310df0-6834-4899-811e-5faf13b2a99c,ef1756b6-354c-4ea4-b175-4eba835b60a5,88f94e8a-0145-489f-9f1d-d4755456e965,cash_out_type_3,4.470917e-05,0.000346,NaN,2018-10-31 12:06:08,NaN,>= 60,5.0,1.530573e+09


In [130]:
new_processed_labeled.isna().sum()

operation_id                 169
request_id                     0
account_id                     0
device_id                      0
counterparty_document          0
operation_type                 0
amount                         0
balance                    53960
requested_at              544491
processed_at                   0
is_fraud                  939400
age_range                    163
number_of_selfies_sent         4
oldest_inserted_at             0
dtype: int64

In [22]:
new_processed_labeled.to_csv("processed_labeled_with_extra.csv", index=False)

In [10]:
new_processed_labeled = pd.read_csv("processed_labeled_with_extra.csv")

In [45]:
test_merge(10)

,operation_id,request_id,account_id,device_id,counterparty_document,operation_type,amount,balance,requested_at,processed_at,is_fraud
737038,94c217bb-8bcc-43db-af42-6f6a5c9e159a,e3099082-d354-47cc-ac55-8f4c129a9c1c,592ef3c0-9f4a-4632-af03-9c6bb887e5e3,d0327873-4e6b-4cdd-b4a4-b4a48e595a28,9e969e22-e7fe-429a-8939-07356030e6a0,cash_out_type_2,1.826061e-06,NaN,NaN,2018-10-18 14:11:36,NaN
153683,0d915790-a446-437e-9700-94abc585f539,7d919f8b-dc57-4526-a66f-ef3a60fb2b96,e30fb3c5-134e-4c88-bef1-44c710d6dc72,57ec3138-72b3-4bde-9b0c-64557a524956,15a5bc20-b469-4f52-b791-27da8372f2fe,cash_out_type_3,4.560079e-07,0.000003,2018-11-15 21:54:26,2018-11-15 21:54:29,NaN
492942,b2ae12b4-6325-42a6-9de4-fcf26f82a914,67326ca8-b0de-4175-b159-0cc4d8eb22fa,dfe8aadf-d935-4e53-bff7-0f54688b333a,0f657de1-c349-472e-8785-2ca2717326d7,ce5f0b6b-dc6f-4a2e-b81c-e968538ede5a,cash_out_type_2,1.623165e-06,0.000002,NaN,2018-11-09 14:50:33,NaN
271143,0b8bbdca-36cf-4c69-896a-35f25c312075,61fe4054-1b47-4244-9586-063f3beea04c,1524028f-562a-4fcd-9e24-7ad75bc1db86,71f7ba3e-0b7a-4d3c-9dd0-5d5e9534283a,787a3b0b-eed7-4459-a9cf-3bb4319cf70b,cash_out_type_2,4.057913e-06,0.000047,2018-11-19 11:50:57,2018-11-19 11:50:57,NaN
878877,668fe230-e705-4e7a-9e48-ceb81adb7e8a,56f46c15-975f-4f2d-af60-ae9aa7a7b007,bb2fb693-adb7-47af-9069-360f076e05b4,afc7c4e4-d9f3-4429-b0e9-3ff357068025,643ccbe1-31c4-4783-8781-7bef618839ee,cash_out_type_2,3.997044e-06,0.000084,NaN,2018-10-31 09:30:46,NaN
41592,4da7d3d0-19e7-49ca-a182-2458fd0be27f,8ee2efcc-9a42-4a05-8b2c-e590516e0c25,d7906b3d-5d99-4fc5-8cbf-0450fe44a8a2,5dea1e55-843f-410d-8ed6-283f8ef2e53d,949a409c-830c-4cf7-a2b1-b30b424d5aa7,cash_out_type_3,2.191273e-06,NaN,NaN,2018-10-19 18:18:23,NaN
687164,7d352ba6-9d39-467c-b1bc-c0d76307f06e,f36d261f-5838-47f8-aa9a-f34db654ada0,89883809-0208-41e0-9378-af47d88d155c,3290fb3e-8b30-4071-98e9-df02e22f689c,48f282fc-8280-4007-adf1-adcbd0a9b94a,cash_out_type_2,1.948011e-05,0.000019,2018-11-14 15:20:56,2018-11-14 15:20:56,NaN
449065,96861750-0657-4a4e-9c3e-f6ca96b467c9,a3b545bf-eb2d-4663-abb8-39453ac7dcac,2329b1fb-fb89-4ee3-a5f8-7fbfdfb9ece9,efba1cec-7903-47ea-ba7f-1cf24e01c771,5033488c-125e-4b63-a2b1-1bd5b2f9436f,cash_out_type_2,4.950654e-06,0.000007,NaN,2018-10-23 18:17:32,NaN
31152,42f622b2-9477-4a18-90a5-683d13bc11d8,e478bb18-dc67-4f5b-869e-02440c1c56d6,c2fd6a8e-a9a9-46ee-a571-6bb7c654ee46,1ee541b5-6491-4347-ad72-a50fd8e03ba7,c1b2cf9e-2444-4c14-9746-2163875f7beb,cash_out_type_3,4.463704e-06,0.000082,NaN,2018-10-24 22:26:47,NaN
568742,ce80140d-c151-474c-b633-ddf2f8794a0d,79c20d42-440f-4a0c-b3ef-96019bf97a49,4e877d8b-aa3c-416a-8a6e-3bccf6b9fb11,38622c65-664c-40bb-a930-e57a4fa1327f,4f05ddde-aaaf-4b73-b399-15cc0ff51ba1,cash_out_type_3,1.314967e-06,0.000001,NaN,2018-10-23 13:20:38,NaN


,id,owner_id,owner_document,owner_type,inserted_at
271828,bb2fb693-adb7-47af-9069-360f076e05b4,96226b06-0802-49ce-bbb4-7d276f71a5d2,75313d34-7002-4551-a1d6-cd4de5aed855,organization,2018-10-24 12:53:39.693080


Find organization 96226b06-0802-49ce-bbb4-7d276f71a5d2? y


,user_id,organization_id
43742,ca5734e0-7fb9-49b4-b03c-9e771b78f757,96226b06-0802-49ce-bbb4-7d276f71a5d2


,id,organization_id,type,value,inserted_at,updated_at
226263,ae3c4438-9568-41c6-a308-d5ca55f0c9c5,96226b06-0802-49ce-bbb4-7d276f71a5d2,organization_type,ME,2018-10-24 12:53:30.636984,2018-10-24 12:53:30.636984


...

,id,document,inserted_at
49813,ca5734e0-7fb9-49b4-b03c-9e771b78f757,27bf0210-cf0b-48c6-b3c3-239c2855e2ca,2018-10-24 12:53:29.477666


Show account trail for account id bb2fb693-adb7-47af-9069-360f076e05b4 and merge dataframe? y


{'account_id': 'bb2fb693-adb7-47af-9069-360f076e05b4',
 'age_range': '25-39',
 'number_of_selfies_sent': 5,
 'oldest_inserted_at': datetime.datetime(2018, 10, 24, 12, 53, 29, 477666)}

,operation_id,request_id,account_id,device_id,counterparty_document,operation_type,amount,balance,requested_at,processed_at,is_fraud,age_range,number_of_selfies_sent,oldest_inserted_at
878863,bffa3376-0d58-4f96-9796-d627d4c27706,308029a0-c3c3-4afe-939e-a74b6bd351fe,bb2fb693-adb7-47af-9069-360f076e05b4,afc7c4e4-d9f3-4429-b0e9-3ff357068025,18373932-fd76-47e8-8ac3-09a98c9a650c,cash_out_type_2,0.000003,0.000091,NaN,2018-10-31 09:30:42,NaN,25-39,5.0,1.540396e+09
878864,e1ae08fa-b935-47b9-8827-77dc2f3748ee,79ce2977-1e01-4165-a5af-e649bb4a31e7,bb2fb693-adb7-47af-9069-360f076e05b4,afc7c4e4-d9f3-4429-b0e9-3ff357068025,18373932-fd76-47e8-8ac3-09a98c9a650c,cash_out_type_2,0.000004,0.000052,2018-11-14 01:34:59,2018-11-14 12:00:05,NaN,25-39,5.0,1.540396e+09


KeyboardInterrupt: Interrupted by user

In [51]:
new_processed_labeled.groupby("number_of_selfies_sent")["number_of_selfies_sent"].count()

number_of_selfies_sent
1.0       2140
2.0       2128
3.0       1714
4.0        609
5.0     615894
6.0     287370
7.0        190
8.0        133
9.0        148
10.0     17366
11.0      5094
12.0       784
13.0       307
14.0       292
15.0      2203
16.0      1336
17.0       632
18.0       326
19.0       199
20.0       627
21.0       267
22.0       118
23.0        97
24.0        66
25.0       173
26.0        72
27.0        41
28.0       190
29.0        98
30.0        70
31.0        10
33.0         8
34.0         8
35.0        30
36.0        12
38.0        23
41.0        20
42.0        10
43.0       103
48.0         3
56.0        20
Name: number_of_selfies_sent, dtype: int64